In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_classic.output_parsers import EnumOutputParser

In [2]:
model=ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [32]:
class LLMState(TypedDict):
    topic:str
    outline:str
    blog:str
    eval_score:int

In [47]:
def evaluate_blog(state:LLMState)->LLMState:
    outline=state['outline']
    blog=state['blog']
    prompt=PromptTemplate(
        template='''Based on the given outline evaluate my whole blog and rate it out of 10 in integer and only return the score and nothing else.
        The Outline is given below:
        {outline}
        
        The blog is given below:
        {blog}
        ''',
        input_variables=['outline','blog']
    )
    formatted_prompt=prompt.format(outline=outline,blog=blog)
    eval_score=model.invoke(formatted_prompt).content
    state['eval_score']=eval_score
    
    return state

In [48]:
def generate_outline(state:LLMState)->LLMState:
    topic=state["topic"]
    prompt=PromptTemplate(
        template="Please generate a detailed outline to write a blog on {topic}",
        input_variables=['topic']
    )
    generated_outline=model.invoke(prompt.format(topic=topic)).content
    state['outline']=generated_outline
    
    return state    
    

In [49]:
def generate_blog(state:LLMState)->LLMState:
    outline=state['outline']
    topic=state['topic']
    prompt=PromptTemplate(
        template='''Based on the given outline text generate a detailed blog on {topic}:
        {outline}
        ''',
        input_variables=['topic','outline']
    )
    generated_blog=model.invoke(prompt.format(outline=outline,topic=topic)).content
    state['blog']=generated_blog
    
    return state

In [50]:
graph=StateGraph(LLMState)

#add nodes
graph.add_node('generate_outline',generate_outline)
graph.add_node('generate_blog',generate_blog)
graph.add_node('eval_score',evaluate_blog)

#add edge
graph.add_edge(START,'generate_outline')
graph.add_edge('generate_outline','generate_blog')
graph.add_edge('generate_blog','eval_score')
graph.add_edge('eval_score',END)

#compile
workflow=graph.compile()

In [51]:
initial_state={'topic':'Eminem'}

output=workflow.invoke(initial_state)

In [52]:
print(output['outline'])

Here's a detailed outline for a blog post about Eminem, covering his career, impact, and artistry.

---

## Blog Title Options:

*   **Eminem: Beyond the Shock Value – Unpacking the Genius of a Rap Icon**
*   **The Marshall Mathers Story: How Eminem Redefined Hip-Hop and Himself**
*   **Rap God to Cultural Phenomenon: The Enduring Legacy of Eminem**
*   **More Than Just Rhymes: Exploring the Complex World of Eminem**

---

## Blog Post Outline:

**I. Introduction: The Unmistakable Roar of Shady**

*   **A. Hook:** Start with a provocative statement or a vivid description of Eminem's impact.
    *   *Example:* "Few artists ignite as much debate, controversy, and fervent loyalty as Marshall Mathers III, better known as Eminem. From his groundbreaking emergence to his continued reign, his presence in music is undeniably seismic."
*   **B. Brief Overview:** Introduce Eminem – his multiple personas (Eminem, Slim Shady, Marshall Mathers), his status as a global icon, and his unique position 

In [53]:
print(output['blog'])

## Eminem: Beyond the Shock Value – Unpacking the Genius of a Rap Icon

Few artists ignite as much debate, controversy, and fervent loyalty as Marshall Mathers III, better known to the world as Eminem. From his groundbreaking emergence in the late 90s to his continued reign as a global phenomenon, his presence in music is undeniably seismic. Whether he’s embodying the chaotic id of Slim Shady, the introspective vulnerability of Marshall Mathers, or the technical mastery of Eminem, he stands as a unique and often polarizing figure in hip-hop history. This blog post aims to delve beyond the headlines and explore the multifaceted genius, the controversial persona, the unparalleled lyrical prowess, and the undeniable cultural impact of Eminem, demonstrating precisely why he remains a crucial and enduring figure in music.

### The Origin Story: From Detroit Underdog to Dr. Dre's Protégé

Eminem's journey to stardom is as compelling and gritty as the streets of Detroit that shaped him. Growi

In [54]:
print(output['topic'])

Eminem


In [55]:
print(output['eval_score'])

10


In [56]:
print(output)

{'topic': 'Eminem', 'outline': 'Here\'s a detailed outline for a blog post about Eminem, covering his career, impact, and artistry.\n\n---\n\n## Blog Title Options:\n\n*   **Eminem: Beyond the Shock Value – Unpacking the Genius of a Rap Icon**\n*   **The Marshall Mathers Story: How Eminem Redefined Hip-Hop and Himself**\n*   **Rap God to Cultural Phenomenon: The Enduring Legacy of Eminem**\n*   **More Than Just Rhymes: Exploring the Complex World of Eminem**\n\n---\n\n## Blog Post Outline:\n\n**I. Introduction: The Unmistakable Roar of Shady**\n\n*   **A. Hook:** Start with a provocative statement or a vivid description of Eminem\'s impact.\n    *   *Example:* "Few artists ignite as much debate, controversy, and fervent loyalty as Marshall Mathers III, better known as Eminem. From his groundbreaking emergence to his continued reign, his presence in music is undeniably seismic."\n*   **B. Brief Overview:** Introduce Eminem – his multiple personas (Eminem, Slim Shady, Marshall Mathers), 